In [ ]:
import numpy  as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(np.array(predictions),np.array(labels))]
    return sum(differences) / len(differences)

def MAE(predictions, labels):
    differences = [abs(x - y) for x, y in zip(predictions, labels)]
    return sum(differences) / len(differences)


def RMSE(predictions, labels):
    differences = [(x - y) ** 2 for x, y in zip(predictions, labels)]
    return (sum(differences) ** 0.5) / len(differences)
  

In [ ]:
df = pd.read_pickle("../Datasets/USA_data_final_prep.pkl")
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df2 = df[df["year_2001"] != 1].drop(columns=["year_2001"])

In [ ]:
dfs

In [ ]:
df2

In [ ]:
df2.columns

In [ ]:
X = np.array(df2.drop(columns=["rating"]))
y = np.array(df2[["rating"]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [ ]:
len(X_train),len(X_test),len(X_val)

In [ ]:
clms = list(df2.columns)
print(len(clms))
clms.remove("rating")

In [ ]:
len(clms)

In [ ]:
len(X_train)

In [ ]:
dataset_dir = "../Datasets/final/"

In [ ]:
np_x = pd.DataFrame(X_train,columns = clms)
np_x.to_pickle(dataset_dir+"X_train.pkl")

np_x = pd.DataFrame(X_val,columns = clms)
np_x.to_pickle(dataset_dir+"X_val.pkl")

np_x = pd.DataFrame(X_test,columns = clms)
np_x.to_pickle(dataset_dir+"X_test.pkl")




In [ ]:
np_y = pd.DataFrame(y_train,columns = ["rating"])
np_y.to_pickle(dataset_dir+"y_train.pkl")

np_y = pd.DataFrame(y_test,columns = ["rating"])
np_y.to_pickle(dataset_dir+"y_test.pkl")


np_y = pd.DataFrame(y_val,columns = ["rating"])
np_y.to_pickle(dataset_dir+"y_val.pkl")



In [ ]:
import os
print(os.getcwd())

In [ ]:
X_train = pd.read_pickle(dataset_dir+"X_train.pkl")
y_train = pd.read_pickle(dataset_dir+"y_train.pkl")

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
X = pd.DataFrame()
X["user"] = X_train["gPlusUserId"]
X["id"] = X_train["gPlusPlaceId"]
X["review"] = y_train["rating"]

In [ ]:
len(X)

In [ ]:
X.user[0]

In [ ]:
from collections import defaultdict

In [ ]:
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
users = set()
items = set()
summ = 0
for i in range(len(X)):
    u = X.user[i]
    b = X.id[i]
    r = X.review[i]
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))
    users.add(u)
    items.add(b)
    summ = summ+ r
summ /= len(X)

In [ ]:
print(summ)

In [ ]:
def train(lmd):
    alpha = summ
    Beta_u = defaultdict(float)
    Beta_i = defaultdict(float)
    flag = True
    prev = 10000
    Flag = True
    while Flag==True:
        a_=0
        for i in range(len(X)):
            u = X.user[i]
            b = X.id[i]
            r = X.review[i]
            a_+= r - (Beta_u[u] - Beta_i[b])
        alpha = a_ / len(X)
        for u in ratingsPerUser:
            b_= 0 
            for (b,r) in ratingsPerUser[u]:
                b_+= r - alpha - Beta_i[b]
            Beta_u[u] = b_ / (lmd + len(ratingsPerUser[u]))
        for i in ratingsPerItem:
            i_=0
            for(u,r) in  ratingsPerItem[i]:
                i_+= r - alpha - Beta_u[u]
            Beta_i[i] = i_ / (lmd + len(ratingsPerItem[i]))
        mse  = 0
        for i in range(len(X)):
            u = X.user[i]
            b = X.id[i]
            r = X.review[i]
            mse+= (r - alpha - Beta_u[u] - Beta_i[b])**2
        obj = 0
        for u in Beta_u:
            obj+= Beta_u[u]**2
        for i in Beta_i:
            obj+= Beta_i[i]**2
        if abs(mse + lmd*obj - prev ) < 0.001:
            Flag = False
        #print(mse)
        #print(lmd*obj)
        print("Loss was ",prev)
        prev = mse + lmd*obj
    return alpha , Beta_u , Beta_i

In [ ]:
al , bu , bi = train(1)

In [ ]:
def average_u(usr):
    s = 0.0 
    c = 0.0
    for i in range(len(X)):
        u = X.user[i]
        b = X.id[i]
        r = X.review[i]
        if usr == u:
            s+=r
            c+=1
    return s/c
def average_i(itm):
    s = 0.0 
    c = 0.0
    for i in range(len(X)):
        u = X.user[i]
        b = X.id[i]
        r = X.review[i]
        if b == itm:
            s+=r
            c+=1
    return s/c
def average_dt():
    return summ

In [ ]:
dic_global_i = {}
dic_global_u = {}

In [ ]:
def pred2(u,i,al,bu,bi):
    p = 0 
    if u in bu and i in bi:
        p = al + bu[u] + bi[i]
    else:
        p = summ
#     elif u not in bu and i not in bi:
#         p =  average_dt()
#     elif u not in bu:
#         if i in dic_global_i:
#             p = dic_global_i[i]
#         else:
#             p = average_i(i)
#             dic_global_i[i] = p
#     else:
#         if u in dic_global_u:
#             p = dic_global_u[u]
#         else:
#             p = average_u(u)
#             dic_global_u[u] = p
        
    p = min(p,5)
    p = max(p,0)
    return p 

In [ ]:
# Performance on training 
pred = []
y = []
for i in range(len(X)):
    u = X.user[i]
    b = X.id[i]
    r = X.review[i]
    y.append(r)
    pred.append(pred2(u,b,al,bu,bi))    

In [ ]:
len(pred),len(y)

In [ ]:
MSE(pred,y)

In [ ]:
MAE(pred,y)

In [ ]:
RMSE(pred,y)

In [ ]:
# Validation set testing 
X_val = pd.read_pickle(dataset_dir+"X_val.pkl")
y_val = pd.read_pickle(dataset_dir+"y_val.pkl")
X_v = pd.DataFrame()
X_v["user"] = X_val["gPlusUserId"]
X_v["id"] = X_val["gPlusPlaceId"]
X_v["review"] = y_val["rating"]

In [ ]:
pred = []
y = []
for i in tqdm(range(len(X_v))):
    u = X_v.user[i]
    b = X_v.id[i]
    r = X_v.review[i]
    y.append(r)
    pred.append(pred2(u,b,al,bu,bi))  
print(MAE(pred,y))
print(MSE(pred,y))

In [ ]:
lamdas = [0.1,0.5,1,2,5,10]
for lmd in lamdas:
    al , bu , bi = train(lmd)
    Pr = []
    y_valid = []
    for i in tqdm(range(len(X_v))):
        u = X_v.user[i]
        b = X_v.id[i]
        r = X_v.review[i]
        y_valid.append(r)
        Pr.append(pred2(u,b,al,bu,bi))
    validMSE = MSE(y_valid,Pr)
    validMAE = MAE(y_valid,Pr)
    print(lmd, "---- ",validMSE," ",validMAE)

In [ ]:
X_test = pd.read_pickle(dataset_dir+"X_test.pkl")
y_test = pd.read_pickle(dataset_dir+"y_test.pkl")
X_t = pd.DataFrame()
X_t["user"] = X_test["gPlusUserId"]
X_t["id"] = X_test["gPlusPlaceId"]
X_t["review"] = y_test["rating"]

In [ ]:
len(X_t)

In [ ]:
lmd = 2
al , bu , bi = train(lmd)
p_t = []
y_train = []
Pr = []
y_valid = []
y_test =[]
pred_test = []
for i in range(len(X)):
    u = X.user[i]
    b = X.id[i]
    r = X.review[i]
    y_train.append(r)
    p_t.append(pred2(u,b,al,bu,bi))
TMSE = MSE(y_train,p_t)
TMAE = MAE(y_train,p_t)
print("Train ---- ",TMSE," ",TMAE)
for i in tqdm(range(len(X_v))):
    u = X_v.user[i]
    b = X_v.id[i]
    r = X_v.review[i]
    y_valid.append(r)
    Pr.append(pred2(u,b,al,bu,bi))
    
validMSE = MSE(y_valid,Pr)
validMAE = MAE(y_valid,Pr)
print("Validation ---- ",validMSE," ",validMAE)

for i in tqdm(range(len(X_t))):
    u = X_t.user[i]
    b = X_t.id[i]
    r = X_t.review[i]
    y_test.append(r)
    pred_test.append(pred2(u,b,al,bu,bi))
    
TMSE = MSE(y_test,pred_test)
TMAE = MAE(y_test,pred_test)
print("Test ---- ",TMSE," ",TMAE)

In [ ]:
# Doing same thing without temporal features 

In [ ]:
X_train = pd.read_pickle("../Datasets/temporal/final/X_train.pkl")
X_val = pd.read_pickle("../Datasets/temporal/final/X_val.pkl")
X_test = pd.read_pickle("../Datasets/temporal/final/X_test.pkl")
y_train = pd.read_pickle("../Datasets/temporal/final/y_train.pkl")
y_val = pd.read_pickle("../Datasets/temporal/final/y_val.pkl")
y_test = pd.read_pickle("../Datasets/temporal/final/y_test.pkl")

In [ ]:
X_train.head()

In [ ]:
X = pd.DataFrame()
X["user"] = X_train["gPlusUserId"]
X["id"] = X_train["gPlusPlaceId"]
X["review"] = y_train["rating"]

In [ ]:
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
users = set()
items = set()
summ = 0
for i in range(len(X)):
    u = X.user[i]
    b = X.id[i]
    r = X.review[i]
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))
    users.add(u)
    items.add(b)
    summ = summ+ r
summ /= len(X)

In [ ]:
al , bu , bi = train(1)

In [ ]:
X_v = pd.DataFrame()
X_v["user"] = X_val["gPlusUserId"]
X_v["id"] = X_val["gPlusPlaceId"]
X_v["review"] = y_val["rating"]

In [ ]:
lamdas = [0.5,1,2,4,5,10]
for lmd in lamdas:
    al , bu , bi = train(lmd)
    Pr = []
    y_valid = []
    for i in tqdm(range(len(X_v))):
        u = X_v.user[i]
        b = X_v.id[i]
        r = X_v.review[i]
        y_valid.append(r)
        Pr.append(pred2(u,b,al,bu,bi))
    validMSE = MSE(y_valid,Pr)
    validMAE = MAE(y_valid,Pr)
    print(lmd, "---- ",validMSE," ",validMAE)

In [ ]:
lamdas = [0.1,0.2,0.3,0.4]
for lmd in lamdas:
    al , bu , bi = train(lmd)
    Pr = []
    y_valid = []
    for i in tqdm(range(len(X_v))):
        u = X_v.user[i]
        b = X_v.id[i]
        r = X_v.review[i]
        y_valid.append(r)
        Pr.append(pred2(u,b,al,bu,bi))
    validMSE = MSE(y_valid,Pr)
    validMAE = MAE(y_valid,Pr)
    print(lmd, "---- ",validMSE," ",validMAE)

In [ ]:
X_t = pd.DataFrame()
X_t["user"] = X_test["gPlusUserId"]
X_t["id"] = X_test["gPlusPlaceId"]
X_t["review"] = y_test["rating"]

In [ ]:
X_t = X_t.reset_index(drop=True)

In [ ]:
lmd = 0.1
al , bu , bi = train(lmd)
p_t = []
y_train = []
Pr = []
y_valid = []
y_test =[]
pred_test = []
for i in range(len(X)):
    u = X.user[i]
    b = X.id[i]
    r = X.review[i]
    y_train.append(r)
    p_t.append(pred2(u,b,al,bu,bi))
TMSE = MSE(y_train,p_t)
TMAE = MAE(y_train,p_t)
print("Train ---- ",TMSE," ",TMAE)
pred_test = []
for i in tqdm(range(len(X_v))):
    u = X_v.user[i]
    b = X_v.id[i]
    r = X_v.review[i]
    y_valid.append(r)
    Pr.append(pred2(u,b,al,bu,bi))
    
validMSE = MSE(y_valid,Pr)
validMAE = MAE(y_valid,Pr)
print("Validation ---- ",validMSE," ",validMAE)



In [ ]:
pred_test = []
for i in tqdm(range(len(X_t))):
    u = X_t.user[i]
    b = X_t.id[i]
    pred_test.append(pred2(u,b,al,bu,bi))
    
TMSE = MSE(y_test["rating"],pred_test)
TMAE = MAE(y_test["rating"],pred_test)
print("Test ---- ",TMSE," ",TMAE)

In [ ]:
summ

In [ ]:
## What happens if we predict mean every time 
pred_test = []
for i in tqdm(range(len(X_t))):
    pred_test.append(summ)
    
TMSE = MSE(y_test["rating"],pred_test)
TMAE = MAE(y_test["rating"],pred_test)
print("Test ---- ",TMSE," ",TMAE)